In [1]:
import numpy as np
import pandas as pd

#### Load the data

In [6]:
DATA_PATH = './data/'
FILE2016 = DATA_PATH + 'quotes-2016.json.bz2'
df_base = pd.read_json(FILE2016, lines=True, compression='bz2', nrows=1000)

#### Create a copy to work with

In [12]:
df = df_base.copy()
df

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2016-12-26-000040,[ ] and Chris [ Jones ] were in there a lot an...,Andy Reid,"[Q2622812, Q27830815, Q470738, Q4761219]",2016-12-26 20:05:00,1,"[[Andy Reid, 0.9432], [None, 0.0541], [Trevor ...",[http://www.kcchiefs.com/news/article-2/How-a-...,E
1,2016-07-31-000006,[ And ] I don't know if we have enough time to...,Mike Howe,[Q6847325],2016-07-31 08:22:12,2,"[[Mike Howe, 0.7118], [None, 0.2882]]",[http://www.peninsuladailynews.com/apps/pbcs.d...,E
2,2016-09-06-000292,... I feel like I was champion long before I l...,None,[],2016-09-06 20:54:45,2,"[[None, 0.6877], [John Waters, 0.3123]]",[http://onlineathens.com/breaking-news/2016-09...,E
3,2016-07-11-000226,[ I ] mmigration has been and continues to be ...,Hillary Clinton,[Q6294],2016-07-11 17:26:06,1,"[[Hillary Clinton, 0.9025], [None, 0.0975]]",[http://www.breitbart.com/tech/2016/07/11/hill...,E
4,2016-05-26-000371,[ It is ] the process of understanding what ki...,Bruce Maxwell,[Q26129591],2016-05-26 15:21:37,1,"[[Bruce Maxwell, 0.8178], [None, 0.1822]]",[http://www.scout.com/mlb/athletics/story/1673...,E
...,...,...,...,...,...,...,...,...,...
995,2016-02-18-024787,Everybody's watching the weather in India. The...,Mike Davey,"[Q6846495, Q6846496]",2016-02-18 10:00:48,3,"[[Mike Davey, 0.8103], [None, 0.1897]]",[http://www.cbc.ca/news/business/pulse-crops-l...,E
996,2016-12-19-016854,Everyone chipped in and did a great job.,None,[],2016-12-19 17:58:57,1,"[[None, 0.9884], [Tom Savage, 0.0116]]",[http://texashsfootball.com/east-blows-west-el...,E
997,2016-07-18-019424,"Everyone has a right to voice their opinion, e...",Norman Kwong,[Q119775],2016-07-18 14:05:38,1,"[[Norman Kwong, 0.7824], [None, 0.2176]]",[http://nbcnews.com/news/asian-america/asian-a...,E
998,2016-08-09-019117,Everyone has been so friendly and I cannot wai...,Guochuan Lai,[Q27975214],2016-08-09 17:14:51,4,"[[Guochuan Lai, 0.8836], [None, 0.1131], [Alan...",[http://expressandstar.com/sport/west-bromwich...,E


#### Remove `None` speakers

In [13]:
df_speaker = df.drop(df[df['speaker']=='None'].index)
df_speaker = df_speaker.reset_index()
df_speaker

,index,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,0,2016-12-26-000040,[ ] and Chris [ Jones ] were in there a lot an...,Andy Reid,"[Q2622812, Q27830815, Q470738, Q4761219]",2016-12-26 20:05:00,1,"[[Andy Reid, 0.9432], [None, 0.0541], [Trevor ...",[http://www.kcchiefs.com/news/article-2/How-a-...,E
1,1,2016-07-31-000006,[ And ] I don't know if we have enough time to...,Mike Howe,[Q6847325],2016-07-31 08:22:12,2,"[[Mike Howe, 0.7118], [None, 0.2882]]",[http://www.peninsuladailynews.com/apps/pbcs.d...,E
2,3,2016-07-11-000226,[ I ] mmigration has been and continues to be ...,Hillary Clinton,[Q6294],2016-07-11 17:26:06,1,"[[Hillary Clinton, 0.9025], [None, 0.0975]]",[http://www.breitbart.com/tech/2016/07/11/hill...,E
3,4,2016-05-26-000371,[ It is ] the process of understanding what ki...,Bruce Maxwell,[Q26129591],2016-05-26 15:21:37,1,"[[Bruce Maxwell, 0.8178], [None, 0.1822]]",[http://www.scout.com/mlb/athletics/story/1673...,E
4,5,2016-02-12-000118,[ It ] would give us more flexibility in terms...,Bob McDonald,"[Q41948618, Q4933333]",2016-02-12 18:30:52,1,"[[Bob McDonald, 0.9298], [None, 0.0702]]",[http://federalnewsradio.com/ses/2016/02/ses-m...,E
...,...,...,...,...,...,...,...,...,...,...
614,994,2016-05-26-022023,Everybody's been supportive (but) everybody's ...,Mike Pennel,[Q17712918],2016-05-26 20:55:46,1,"[[Mike Pennel, 0.551], [None, 0.4184], [Demetr...",[http://www.scout.com/nfl/packers/story/167313...,E
615,995,2016-02-18-024787,Everybody's watching the weather in India. The...,Mike Davey,"[Q6846495, Q6846496]",2016-02-18 10:00:48,3,"[[Mike Davey, 0.8103], [None, 0.1897]]",[http://www.cbc.ca/news/business/pulse-crops-l...,E
616,997,2016-07-18-019424,"Everyone has a right to voice their opinion, e...",Norman Kwong,[Q119775],2016-07-18 14:05:38,1,"[[Norman Kwong, 0.7824], [None, 0.2176]]",[http://nbcnews.com/news/asian-america/asian-a...,E
617,998,2016-08-09-019117,Everyone has been so friendly and I cannot wai...,Guochuan Lai,[Q27975214],2016-08-09 17:14:51,4,"[[Guochuan Lai, 0.8836], [None, 0.1131], [Alan...",[http://expressandstar.com/sport/west-bromwich...,E


In [19]:
print('In total,', len(df)-len(df_speaker), 'rows had no speaker and were thus removed')

In total, 381 rows had no speaker and were thus removed


#### Remove speakers for which the probability is smaller than a threshold

In [1]:
#Define threshold (50%)
threshold_probas = 0.5

#Gather the first probability for each row
string_probas = [df_speaker['probas'][i][0][1] for i in range(len(df_speaker))]
nber_probas = list(map(float, string))
series_probas = pd.Series(nber_probas)

#Check if the probability is larger than the threshold, if not remove the row
df_probas = df_speaker.drop(series_probas[series_probas < threshold_probas].index)
df_probas

NameError: name 'df_speaker' is not defined

In [25]:
print('In total,', len(df_speaker)-len(df_probas), 'speakers had a probality smaller than', threshold_probas, 'and were thus removed')

In total, 34 speakers had a probality smaller than 0.5 and were thus removed


#### Think to other scenarios

In [ ]:
#If all speakers are women but with proba < threshold, keep the row ?
#might cause some problem for interpretation (country of origin of the speaker)


#### Additional notes Lavi 1.11.

- I don't think we should keep the speakers if proba < threshold, because like you state above, this will create additional problems


- It would be good to make a function for the remove None speakers such that it can be used when processing in batches once we clean the WHOLE data set
- Same for proba


Possible additional cleaning may include:
- checking nan values 
- verify absurd QuoteID or QID 
- make sure quotes are non empty

